In [3]:
import numpy as np 
import pandas as pd 

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import tensorflow as tf
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [4]:
# lines=pd.read_csv("/content/drive/MyDrive/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')
url = 'https://raw.githubusercontent.com/prismspeechproject/neural/master/implementation/English_hindi.csv'
lines=pd.read_csv(url, encoding='utf-8')
lines= lines[lines.columns[0:2]]
lines=lines[~pd.isnull(lines['English'])]
lines=lines[~pd.isnull(lines['Hindi'])]
lines.drop_duplicates(inplace=True)
lines=lines.sample(n=25000,random_state=42)
lines.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(25000, 2)

In [5]:
lines['English']=lines['English'].apply(lambda x: x.lower())
lines['Hindi']=lines['Hindi'].apply(lambda x: x.lower())

In [6]:

lines['English']=lines['English'].apply(lambda x: re.sub("'", '', x))
lines['Hindi']=lines['Hindi'].apply(lambda x: re.sub("'", '', x))

In [7]:
exclude = set(string.punctuation) 
lines['English']=lines['English'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['Hindi']=lines['Hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [8]:
remove_digits = str.maketrans('', '', digits)
lines['English']=lines['English'].apply(lambda x: x.translate(remove_digits))
lines['Hindi']=lines['Hindi'].apply(lambda x: x.translate(remove_digits))

lines['Hindi'] = lines['Hindi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

lines['English']=lines['English'].apply(lambda x: x.strip())
lines['Hindi']=lines['Hindi'].apply(lambda x: x.strip())
lines['English']=lines['English'].apply(lambda x: re.sub(" +", " ", x))
lines['Hindi']=lines['Hindi'].apply(lambda x: re.sub(" +", " ", x))

In [9]:

lines['Hindi'] = lines['Hindi'].apply(lambda x : 'START_ '+ x + ' _END')
lines.head()

,English,Hindi
93994,it is the land of festivals of different religions like diwali dushehra pongal and onam idulfitr muhrram chrimas ester are very much famous,START_ विभिन्न धर्मों के इस भूभाग पर कई मनभावन पर्व त्यौहार मनाए जाते हैं दिवाली होली दशहरा पोंगल तथा ओणम ईदउलफितर मुहर्रम क्रिसमस ईस्टर आदि भी काफ़ी लोकप्रिय हैं। _END
124301,enceladus seen here,START_ एनसेलेडस पर इस जगह _END
28869,ghanaian businessmen south african enterprising leaders,START_ या फिर घाना के व्यवसायियों से या दक्षिण अफ्रिका के उद्योगों में अग्रणी भूमिका निभाने वालों से _END
54938,the burma oil company processed the crude in its refinery at rangoon and distributed the products mainly kerosene through its tank installations at chief indian ports and through a fleet of steam tankers,START_ बर्मा आयल कंपनी अपनी रंगून की रिफाइनरी में कच्चे तेल को साफ करती थी और अपने उत्पादन का वितरण जो मुख़्यत मिट्टी का तेल था प्रधान भारतीय बंदरगाहों पर लगे टैंक प्रतिष्ठानों के द्वारा तथा स्टीम टैंकरों के द्वारा करती थी _END
49997,ismail afandi aka ismail khan of the ottoman empire designer of the main dome,START_ मुख्य गुम्बद का अभिकल्पक इस्माइल एकाइस्माइल खाँ जो कि ऑट्टोमन साम्राज्य का प्रमुख गोलार्ध एवं गुम्बद अभिकल्पक थे। _END


In [10]:

all_eng_words=set()
for eng in lines['English']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['Hindi']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [11]:
len(all_eng_words)

30829

In [12]:
len(all_hindi_words)

37434

In [13]:
lines['length_eng_sentence']=lines['English'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['Hindi'].apply(lambda x:len(x.split(" ")))

In [14]:
lines.head()

,English,Hindi,length_eng_sentence,length_hin_sentence
93994,it is the land of festivals of different religions like diwali dushehra pongal and onam idulfitr muhrram chrimas ester are very much famous,START_ विभिन्न धर्मों के इस भूभाग पर कई मनभावन पर्व त्यौहार मनाए जाते हैं दिवाली होली दशहरा पोंगल तथा ओणम ईदउलफितर मुहर्रम क्रिसमस ईस्टर आदि भी काफ़ी लोकप्रिय हैं। _END,23,30
124301,enceladus seen here,START_ एनसेलेडस पर इस जगह _END,3,6
28869,ghanaian businessmen south african enterprising leaders,START_ या फिर घाना के व्यवसायियों से या दक्षिण अफ्रिका के उद्योगों में अग्रणी भूमिका निभाने वालों से _END,6,19
54938,the burma oil company processed the crude in its refinery at rangoon and distributed the products mainly kerosene through its tank installations at chief indian ports and through a fleet of steam tankers,START_ बर्मा आयल कंपनी अपनी रंगून की रिफाइनरी में कच्चे तेल को साफ करती थी और अपने उत्पादन का वितरण जो मुख़्यत मिट्टी का तेल था प्रधान भारतीय बंदरगाहों पर लगे टैंक प्रतिष्ठानों के द्वारा तथा स्टीम टैंकरों के द्वारा करती थी _END,33,43
49997,ismail afandi aka ismail khan of the ottoman empire designer of the main dome,START_ मुख्य गुम्बद का अभिकल्पक इस्माइल एकाइस्माइल खाँ जो कि ऑट्टोमन साम्राज्य का प्रमुख गोलार्ध एवं गुम्बद अभिकल्पक थे। _END,14,20


In [15]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [16]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [17]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [18]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(30829, 37434)

In [19]:
num_decoder_tokens += 1 
num_encoder_tokens = num_encoder_tokens + 1 

In [20]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [21]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [22]:
lines = shuffle(lines)
lines.head(10)

,English,Hindi,length_eng_sentence,length_hin_sentence
20628,he received nobel prize for literature in for his poetical composition ‘geetanjali,START_ उनकी काव्यरचना गीतांजलि के लिये उन्हे सन में साहित्य का नोबेल पुरस्कार मिला। _END,12,15
46802,reduce the incidence of invasive cervical cancer by approximately,START_ इनवेसिव सरवाइकल ह्ययोनि ग्रीवा संबंधी कैंसरहृ की घटनाओं में लगभग प्रतिशत तक कमी _END,9,15
114466,rajasthan is bharats very important region,START_ राजस्थान भारत का एक महत्ती प्रांत है। _END,6,9
6844,the leg muscles may get cramps weakness or discomfort on walking,START_ इसके कारण पैरों की मांसपेशियों में तेज दर्द या कमजोरी अथवा चलने पर तकलीफ होने लगती है _END,11,19
106869,in openness and inclusiveness,START_ एक अभ्यास शुरू किया _END,4,6
30146,every shrine in tajmahal leads to a broad mehrabi basement,START_ ताजमहल में हरेक पुण्यस्थान एक वृहत मेहराबी तहखाने में खुलता है। _END,10,13
80340,and its a little bit like a peace corps for geeks,START_ यूँ समझिये जैसे कंप्यूटर की दुनिया में ही रहनेवाले विचित्र विशेषज्ञ को शान्ति सेना में भेज दिया जाए _END,11,20
104378,hussain a curious mix of bottle and whine in india said it over and over again,START_ भारत में खासे चिड़ेचिड़ै दिखे ह्सैन ने यह बात बारबार कही _END,16,13
57390,there was jhasi ki rani regiment for ladies in azad hind army,START_ आज़ाद हिन्द फ़ौज में औरतो के लिए झाँसी की रानी रेजिमेंट भी बनायी गयी। _END,12,16
84787,normality overlooks the beauty that differences give us,START_ साधारणता असाधारण सुंदरता को अनदेखा करती है _END,8,9


In [23]:
X, y = lines['English'], lines['Hindi']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((13168,), (3293,))

In [24]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [25]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [26]:
latent_dim=300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [27]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [30]:
train_samples = len(X_train)
val_samples = len(X_test)
epochs = 100
batch_size =128
# checkpoint_path = "/content/drive/MyDrive/Colab/ResultPNBy/" /training_2/cp-{epoch:04d}.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# cp_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_path, 
#     verbose=1, 
#     save_weights_only=True,
#     save_freq=5*batch_size)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()


model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    9249000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    11230500    input_2[0][0]                    
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
102/102 [==============================] - 67s 407ms/step - loss: 4.0810 - val_loss: 3.4942
Epoch 2/100
102/102 [==============================] - 44s 426ms/step - loss: 3.4844 - val_loss: 3.4377
Epoch 3/100
102/102 [==============================] - 39s 381ms/step - loss: 3.3530 - val_loss: 3.3334
Epoch 4/100
102/102 [==============================] - 43s 424ms/step - loss: 3.2081 - val_loss: 3.2685
Epoch 5/100
102/102 [==============================] - 40s 385ms/step - loss: 3.0842 - val_loss: 3.2218
Epoch 6/100
102/102 [==============================] - 39s 382ms/step - loss: 2.9793 - val_loss: 3.2017
Epoch 7/100
102/102 [==============================] - 40s 394ms/step - loss: 2.8848 - val_loss: 3.1833
Epoch 8/100
102/102 [==============================] - 43s 421ms/step - loss: 2.7981 - val_loss: 3.1845
Epoch 9/100
102/102 [==============================] - 39s 383ms/step - loss: 2.7106 - val_loss: 3.1495
Epoch 10/100
102/102 [==============================] - 44s 426m

Mounted at /content/drive


In [31]:
import keras
model.save('/content/drive/MyDrive/Colab/ResultPNA2.h5')
# model = keras.models.load_model('/content/drive/MyDrive/ColabNotebooks/ResultPN.h5')
new_model = keras.models.load_model('/content/drive/MyDrive/Colab/ResultPNA2.h5')

In [59]:
# new_model.get_weights()
decoder_model.save_weights('/content/drive/MyDrive/Colab/ResultPNA2DecoderModelW.h5')
# decoder_model.save('/content/drive/MyDrive/Colab/ResultPNA2DecoderModel.h5')
encoder_model.save_weights('/content/drive/MyDrive/Colab/ResultPNA2EncoderModelW.h5')

# import keras
# encoder1_model = keras.loa('/content/drive/MyDrive/Colab/ResultPNA2EncoderModelW.h5')
# decoder1_model = keras.Model.load_weights(new_model, filepath='/content/drive/MyDrive/Colab/ResultPNA2DecoderModelW.h5')

In [60]:
encoder_model.get_weights()

[array([[-0.03819279, -0.0322782 , -0.03939082, ...,  0.00416565,
          0.02199462, -0.04040111],
        [-0.24462663,  0.07550556, -0.02188983, ..., -0.05438171,
         -0.05992344, -0.14644411],
        [-0.02854334,  0.01091386,  0.06664548, ..., -0.00438381,
          0.09545403, -0.08324859],
        ...,
        [-0.01087225,  0.05336149,  0.09070357, ..., -0.01562318,
         -0.02250707, -0.18338294],
        [-0.1053583 ,  0.04338017, -0.00409952, ...,  0.06310016,
          0.02308963,  0.11768317],
        [-0.00641337, -0.00235736, -0.00291197, ...,  0.06093942,
         -0.07201329, -0.03548596]], dtype=float32),
 array([[-2.3507608e-01, -5.9964079e-02,  3.3216095e-01, ...,
          1.4458965e-01,  2.2424659e-01, -2.0234863e-01],
        [ 1.1874749e-01, -1.2757336e-01,  2.1041945e-02, ...,
          2.0338273e-05, -2.1128204e-01,  2.9830477e-01],
        [-7.7597684e-01, -2.7464086e-01, -8.0915622e-02, ...,
         -2.5459874e-01, -1.7398483e-01, -1.4230114e-01]

In [62]:
# Encode the input sequence to get the "thought vectors"
encoder1_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) 

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder1_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [63]:
decoder1_model.load_weights('/content/drive/MyDrive/Colab/ResultPNA2DecoderModelW.h5')
encoder1_model.load_weights('/content/drive/MyDrive/Colab/ResultPNA2EncoderModelW.h5')


In [64]:
def decode_sequence(input_seq):
    states_value = encoder1_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['START_']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder1_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence

In [65]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [66]:
from nltk.translate.bleu_score import corpus_bleu

In [67]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: in the process he manages to strike a new romantic attitude
Actual Hindi Translation:  इसी दौरान वह अपने में एक नया रोमानी बदलाव पाता है 
Predicted Hindi Translation:  इसी दौरान वह अपने में एक नया रोमानी बदलाव पाता है 


In [44]:
# type(decoded_sentence.split())
y =y_train[k:k+1].values[0][6: -4].split()
Y= decoded_sentence[:-4].split()
# print(type(y), type(Y))
print('BLEU-1: %f' % corpus_bleu(y, Y, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % corpus_bleu(y, Y, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % corpus_bleu(y, Y, weights=(0.33, 0.33, 0.33, 0)))
print('BLEU-4: %f' % corpus_bleu(y, Y, weights=(0.25, 0.25, 0.25, 0.25)))
# type(y_train.to_list())

BLEU-1: 0.974359
BLEU-2: 0.987096
BLEU-3: 0.991465
BLEU-4: 0.993527


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [43]:
y, Y

([['इसी',
   'दौरान',
   'वह',
   'अपने',
   'में',
   'एक',
   'नया',
   'रोमानी',
   'बदलाव',
   'पाता',
   'है']],
 [['इसी',
   'दौरान',
   'वह',
   'अपने',
   'में',
   'एक',
   'नया',
   'रोमानी',
   'बदलाव',
   'पाता',
   'है']])

In [ ]:
help(corpus_bleu)

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
print('BLEU-1: %f' % corpus_bleu([y_train[k:k+1].values[0][6:-4].split()], [decoded_sentence[:-4].split()], weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % corpus_bleu(y_train[k:k+1].values[0][6:-4].split(), decoded_sentence[:-4].split(), weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % corpus_bleu(y_train[k:k+1].values[0][6:-4].split(), decoded_sentence[:-4].split(), weights=(0.33, 0.33, 0.33, 0)))
print('BLEU-4: %f' % corpus_bleu(y_train[k:k+1].values[0][6:-4].split(), decoded_sentence[:-4].split(), weights=(0.25, 0.25, 0.25, 0.25)))

In [47]:
print('Individual 1-gram: %f' % corpus_bleu(y, Y))
print('Individual 2-gram: %f' % corpus_bleu(y, Y))
print('Individual 3-gram: %f' % corpus_bleu(y, Y))
print('Individual 4-gram: %f' % corpus_bleu(y, Y))

Individual 1-gram: 0.993527
Individual 2-gram: 0.993527
Individual 3-gram: 0.993527
Individual 4-gram: 0.993527


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
